DAEN 690- CAPSTONE PROJECT- TEAM PATRIOTS

TEAM MEMBERS

1) SHIVA RAM KAUSHIL PABBA
2) ANDREW BLOOM
3) DODDANAIK BASAVARAJ VAKKUND
4) JAHNAVI JONNALAGADDA
======================
THIS NOTEBOOK CONTAINS ANALYSIS FOR PROBLEM STATEMENT-3

1) Find the frequency of second emergencies by incident by first due . X need to be established in minutes. For example, X may be 20-30 minutes?

=============
1) Input dataset : 'Incidents_by_Due.xlsx'
2) Output datasets: 'incident_secondfrequency_average.csv'

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import csv

In [2]:
# Load the dataset
data = pd.read_excel('./Incidents_by_Due.xlsx')

In [6]:
#Print size, columns and data types
print(data.head(2).to_string())
print(data.shape)
print(data.columns)
print(data.dtypes)

   Incident # Event Type Code      Event Type Event Category  Fire Box  First Due  Battalion       Dispatch Time
0  E180010006             ALS  ALS EMERGENCY               E     43503        435      407.0 2018-01-01 00:04:05
1  E180010007        STILALRM     STILL ALARM              F     43602        436      401.0 2018-01-01 00:05:35
(273989, 8)
Index(['Incident #', 'Event Type Code', 'Event Type', 'Event Category',
       'Fire Box', 'First Due', 'Battalion', 'Dispatch Time'],
      dtype='object')
Incident #                 object
Event Type Code            object
Event Type                 object
Event Category             object
Fire Box                    int64
First Due                   int64
Battalion                 float64
Dispatch Time      datetime64[ns]
dtype: object


In [7]:
# dropping missing values
data = data.dropna()

In [8]:
# Converting data type of battalion from float to int
data["Battalion"] = data.Battalion.astype(int)

In [9]:
print(data.dtypes)

Incident #                 object
Event Type Code            object
Event Type                 object
Event Category             object
Fire Box                    int64
First Due                   int64
Battalion                   int64
Dispatch Time      datetime64[ns]
dtype: object


In [10]:
#obtaining date from date time format
data['dispatched_date'] = [d.date() for d in data['Dispatch Time']]


In [11]:
#print number of days in dataset
print(data.dispatched_date.nunique())
print(data["First Due"].nunique())
#print unique event categories
data["Event Category"].unique()

985
46


array(['E', 'F', 'P', 'T', 'N'], dtype=object)

In [12]:
#dropping firstdue/stations 105, 106 since they dont belong to Fairfax county
data.drop(data[data['First Due'] == 106].index, inplace = True) 
data.drop(data[data['First Due'] == 105].index, inplace = True) 


In [13]:
#dropping test and misc event categories
data.drop(data[data['Event Category'] == "T"].index, inplace = True) 

data.drop(data[data['Event Category'] == "N"].index, inplace = True) 

In [14]:
#select required columns only for analysis
df = data[["Incident #","Event Category","First Due","Battalion","Dispatch Time","dispatched_date"]]

In [15]:
df['Dispatch_time_diff'] = np.nan 
# Make sure your data is sorted properly
df = df.sort_values(by=['Event Category', 'First Due','Dispatch Time'])


<ipython-input-15-e38fb21d816c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dispatch_time_diff'] = np.nan


In [16]:
df = df.reset_index()

In [17]:
df[0:3]

,index,Incident #,Event Category,First Due,Battalion,Dispatch Time,dispatched_date,Dispatch_time_diff
0,5,E180010090,E,401,402,2018-01-01 00:40:53,2018-01-01,NaN
1,31,E180010380,E,401,402,2018-01-01 02:34:04,2018-01-01,NaN
2,53,E180010501,E,401,402,2018-01-01 03:56:09,2018-01-01,NaN


In [18]:
df = df.drop(columns=["index"])

In [19]:
#  only take diffs where next row is of the same group
df['Dispatch_time_diff'] = np.where(df["First Due"] == df["First Due"].shift(1), df["Dispatch Time"].diff(), 0)

In [20]:
df

,Incident #,Event Category,First Due,Battalion,Dispatch Time,dispatched_date,Dispatch_time_diff
0,E180010090,E,401,402,2018-01-01 00:40:53,2018-01-01,0 days 00:00:00
1,E180010380,E,401,402,2018-01-01 02:34:04,2018-01-01,0 days 01:53:11
2,E180010501,E,401,402,2018-01-01 03:56:09,2018-01-01,0 days 01:22:05
3,E180010519,E,401,402,2018-01-01 04:12:18,2018-01-01,0 days 00:16:09
4,E180010643,E,401,402,2018-01-01 05:55:35,2018-01-01,0 days 01:43:17
...,...,...,...,...,...,...,...
271810,E192072670,P,465,465,2019-07-26 20:34:01,2019-07-26,124 days 01:09:30
271811,E202102333,P,465,465,2020-07-28 20:29:34,2020-07-28,367 days 23:55:33
271812,E193171331,P,466,465,2019-11-13 11:07:11,2019-11-13,0 days 00:00:00
271813,E193181746,P,466,465,2019-11-14 13:35:57,2019-11-14,1 days 02:28:46


In [21]:
#convert diff to pandas time format
df['Dispatch_time_diff'] = pd.to_timedelta(df['Dispatch_time_diff'])

In [22]:
# Function to convert days to hours
def f(x):
    ts = x.total_seconds()
    hours, remainder = divmod(ts, 3600)
    minutes, seconds = divmod(remainder, 60)
    return ('{}:{:02d}:{:02d}').format(int(hours), int(minutes), int(seconds)) 

df['Dispatch_time_diff_hours'] = df['Dispatch_time_diff'].apply(f)


In [23]:
## converting HH.MM.SS to minutes and round of two decimals
df["Dispatch_time_diff_min"]= df['Dispatch_time_diff_hours'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]) + int(x[2])/60)
df["Dispatch_time_diff_min"]= round(df.Dispatch_time_diff_min,2)

In [24]:
df[0:10]

,Incident #,Event Category,First Due,Battalion,Dispatch Time,dispatched_date,Dispatch_time_diff,Dispatch_time_diff_hours,Dispatch_time_diff_min
0,E180010090,E,401,402,2018-01-01 00:40:53,2018-01-01,00:00:00,0:00:00,0.00
1,E180010380,E,401,402,2018-01-01 02:34:04,2018-01-01,01:53:11,1:53:11,113.18
2,E180010501,E,401,402,2018-01-01 03:56:09,2018-01-01,01:22:05,1:22:05,82.08
3,E180010519,E,401,402,2018-01-01 04:12:18,2018-01-01,00:16:09,0:16:09,16.15
4,E180010643,E,401,402,2018-01-01 05:55:35,2018-01-01,01:43:17,1:43:17,103.28
5,E180011319,E,401,402,2018-01-01 14:32:32,2018-01-01,08:36:57,8:36:57,516.95
6,E180011849,E,401,402,2018-01-01 20:05:24,2018-01-01,05:32:52,5:32:52,332.87
7,E180012178,E,401,402,2018-01-01 23:02:59,2018-01-01,02:57:35,2:57:35,177.58
8,E180020485,E,401,402,2018-01-02 06:49:39,2018-01-02,07:46:40,7:46:40,466.67
9,E180020774,E,401,402,2018-01-02 08:47:02,2018-01-02,01:57:23,1:57:23,117.38


In [25]:
# keep only second row in each group since the second row is the second emergency
grouped_df = df.groupby(["Event Category","First Due","dispatched_date"]).nth(1)

In [26]:
grouped_df

Incident #  Battalion  \
Event Category First Due dispatched_date                          
E              401       2018-01-01       E180010380        402   
                         2018-01-02       E180020774        402   
                         2018-01-03       E180030255        402   
                         2018-01-04       E180041276        402   
                         2018-01-05       E180050580        402   
...                                              ...        ...   
P              442       2020-05-19       E201402120        402   
                         2020-06-17       E201692251        402   
                         2020-07-18       E202000867        402   
                         2020-07-22       E202042807        402   
               466       2019-11-14       E193181798        465   

                                               Dispatch Time  \
Event Category First Due dispatched_date                       
E              401       2018-01-01      2018-01-01 02:34:04   
                         2018-01-02      2018-01-02 08:47:02   
                         2018-01-03      2018-01-03 04:40:21   
                         2018-01-04      2018-01-04 13:27:56   
                         2018-01-05      2018-01-05 07:58:12   
...                                                      ...   
P              442       2020-05-19      2020-05-19 18:51:32   
                         2020-06-17      2020-06-17 21:44:42   
                         2020-07-18      2020-07-18 12:05:32   
                         2020-07-22      2020-07-22 22:20:47   
               466       2019-11-14      2019-11-14 13:51:47   

                                         Dispatch_time_diff  \
Event Category First Due dispatched_date                      
E              401       2018-01-01                01:53:11   
                         2018-01-02                01:57:23   
                         2018-01-03                00:02:30   
                         2018-01-04                00:03:19   
                         2018-01-05                06:46:42   
...                                                     ...   
P              442       2020-05-19                07:01:30   
                         2020-06-17                16:10:55   
                         2020-07-18                09:47:00   
                         2020-07-22                05:29:30   
               466       2019-11-14                00:15:50   

                                         Dispatch_time_diff_hours  \
Event Category First Due dispatched_date                            
E              401       2018-01-01                       1:53:11   
                         2018-01-02                       1:57:23   
                         2018-01-03                       0:02:30   
                         2018-01-04                       0:03:19   
                         2018-01-05                       6:46:42   
...                                                           ...   
P              442       2020-05-19                       7:01:30   
                         2020-06-17                      16:10:55   
                         2020-07-18                       9:47:00   
                         2020-07-22                       5:29:30   
               466       2019-11-14                       0:15:50   

                                          Dispatch_time_diff_min  
Event Category First Due dispatched_date                          
E              401       2018-01-01                       113.18  
                         2018-01-02                       117.38  
                         2018-01-03                         2.50  
                         2018-01-04                         3.32  
                         2018-01-05                       406.70  
...                                                          ...  
P              442       2020-05-19                       421.50  
                

In [27]:
# # Calculating min, max, average frequencies for first due and event category
all_together = (grouped_df.groupby(["First Due","Event Category"])
                  .agg({'Dispatch_time_diff_min': [np.mean, np.min, np.max]})
                  .rename(columns={'mean': 'avg_duration_mins', 'amin': 'min_duration_mins', 'amax': 'max_duration_mins'}))

In [28]:
all_together[0:10]

Dispatch_time_diff_min                    \
                              avg_duration_mins min_duration_mins   
First Due Event Category                                            
401       E                          228.482226              1.22   
          F                          297.690910              0.20   
          P                          370.205044              5.32   
402       E                          298.641729              0.65   
          F                          341.837714              0.40   
          P                          356.333699             13.00   
403       E                          250.588936              0.28   
          F                          328.835526              0.47   
          P                          323.083571              4.58   
404       E                          241.056988              0.45   

                                            
                         max_duration_mins  
First Due Event Category                    
401       E                        1266.15  
          F                        1181.28  
          P                        1382.85  
402       E                        1105.82  
          F                        1357.40  
          P                        1123.55  
403       E                        1231.83  
          F                        1396.18  
          P                        1405.52  
404       E                        1310.63

In [29]:
all_together.reset_index().to_csv("./incident_secondfrequency_average.csv")